# Create Folder Structure

In [ ]:
import directory_tree
import pathlib

def valid_directories(src_dir1: str, src_dir2: str, src_dir3: str, dst_dir: str):
    src_dir1 = pathlib.Path(src_dir1)
    src_dir2 = pathlib.Path(src_dir2)
    src_dir3 = pathlib.Path(src_dir3)
    dst_dir = pathlib.Path(dst_dir)

    assert src_dir1.name == "sequences"
    assert src_dir2.name == "sequences"
    assert src_dir3.name == "sequences"
    assert dst_dir.name == "sequences"

    dst_dir.mkdir(parents=True, exist_ok=True)

    for dir in (src_dir1, src_dir2, src_dir3):
        for ds_store_file in dir.glob("**/.DS_Store"):
            ds_store_file.unlink()

    sequences = zip(
        sorted(src_dir1.iterdir()),
        sorted(src_dir2.iterdir()),
        sorted(src_dir3.iterdir()),
    )

    return sequences, dst_dir

## MotionBEV

According to the documentation, the structure should look like this:

```
path_to_KITTI/
├──sequences
    ├── 00/   
    │   ├── calib.txt   # Calibration file.     
    │   ├── poses.txt   # Odometry poses.
    │   ├── velodyne/	# Unzip from KITTI Odometry Benchmark Velodyne point clouds.
    |   |	├── 000000.bin
    |   |	├── 000001.bin
    |   |	└── ...
    │   └── labels/ 	# Unzip from SemanticKITTI label data.
    |       ├── 000000.label
    |       ├── 000001.label
    |       └── ...
    ├── ...
    └── 21/
    └── ...
```

In [ ]:
import os

%load_ext dotenv
%dotenv

data_path = pathlib.Path(os.environ["DATA_PATH"])

SRC_DIR_VELODYNE = data_path / "kitti/SemanticKITTI/data_velodyne/sequences"
SRC_DIR_CALIB_TIMES = data_path / "kitti/SemanticKITTI/data_calib_times/sequences"
SRC_DIR_LABLE_POSE = data_path / "kitti/SemanticKITTI/data_lable_poses/sequences"

def create_symbolic_links(
    dst_dir: str,
    src_dir_velodyne: str = SRC_DIR_VELODYNE,
    src_dir_calib_times: str = SRC_DIR_CALIB_TIMES,
    src_dir_lable_pose: str = SRC_DIR_LABLE_POSE,
):
    sequences, dst_dir = valid_directories(
        src_dir1=src_dir_velodyne,
        src_dir2=src_dir_calib_times,
        src_dir3=src_dir_lable_pose,
        dst_dir=dst_dir,
    )

    for i, (velodyne, calib_times, label_pose) in enumerate(sequences):
        number = f"{i:02}"
        error_msg = lambda x: f"Should be: {number} is {x.name}"
        assert velodyne.name == number, error_msg(velodyne)
        assert calib_times.name == number, error_msg(calib_times)
        assert label_pose.name == number, error_msg(label_pose)

        new_dir = dst_dir / number
        new_dir.mkdir(parents=True, exist_ok=True)

        for dirs in [velodyne, calib_times, label_pose]:
            print(f'{"=" * 100}\nCreating symbolic links for files in: {dirs}')
            for entry in dirs.iterdir():
                link_name = new_dir / entry.name
                if link_name.exists():
                    print(f"\tFile {link_name} already exists, skipping")
                    continue

                print(f"\tFile {link_name} does not exist, creating")
                link_name.symlink_to(entry, target_is_directory=entry.is_dir())


DST_DIR_MOTIONBEV = data_path / "kitti/SemanticKITTI/symlinks/motionbev/sequences"

create_symbolic_links(dst_dir=DST_DIR_MOTIONBEV)

In [ ]:

directory_tree.display_tree(DST_DIR_MOTIONBEV, max_depth=2)

## MotionSeg3D

According to the documentation, the structure should look like this:

```
DATAROOT
├── sequences
│   └── 08
│       ├── calib.txt                       # calibration file provided by KITTI
│       ├── poses.txt                       # ground truth poses file provided by KITTI
│       ├── velodyne                        # velodyne 64 LiDAR scans provided by KITTI
│       │   ├── 000000.bin
│       │   ├── 000001.bin
│       │   └── ...
│       ├── labels                          # ground truth labels provided by SemantiKITTI
│       │   ├── 000000.label
│       │   ├── 000001.label
│       │   └── ...
│       └── residual_images_1               # the proposed residual images
│           ├── 000000.npy
│           ├── 000001.npy
│           └── ...
```

In [ ]:
data_path = pathlib.Path("/home/vscode/data")

SRC_DIR_VELODYNE = data_path / "kitti/SemanticKITTI/data_velodyne/sequences"
SRC_DIR_CALIB_TIMES = data_path / "kitti/SemanticKITTI/data_calib_times/sequences"
SRC_DIR_LABLE_POSE = data_path / "kitti/SemanticKITTI/data_lable_poses/sequences"
SRC_DIR_RESIDUALS = data_path / "kitti/SemanticKITTI/motionseg3d/outputs/sequences"

def validate_residulas(src_dir: str):
    src_dir = pathlib.Path(src_dir)
    assert src_dir.name == "sequences"

    for ds_store_file in src_dir.glob("**/.DS_Store"):
        ds_store_file.unlink()

    return sorted(src_dir.iterdir())

def create_symbolic_links(
    dst_dir: str,
    src_dir_velodyne: str = SRC_DIR_VELODYNE,
    src_dir_calib_times: str = SRC_DIR_CALIB_TIMES,
    src_dir_lable_pose: str = SRC_DIR_LABLE_POSE,
    src_dir_residuals: str = SRC_DIR_RESIDUALS,
):
    sequences, dst_dir = valid_directories(
        src_dir1=src_dir_velodyne,
        src_dir2=src_dir_calib_times,
        src_dir3=src_dir_lable_pose,
        dst_dir=dst_dir,
    )

    sequences = list(sequences)
    residuals = validate_residulas(src_dir_residuals)

    ERROR_MSG = "Number of sequences do not match number of residuals"
    assert len(sequences) == len(residuals), ERROR_MSG

    extended_sequences = zip(sequences, residuals)
    for i, ((velodyne, calib_times, label_pose), residual) in enumerate(extended_sequences):
        number = f"{i:02}"
        error_msg = lambda x: f"Should be: {number} is {x.name}"
        assert velodyne.name == number, error_msg(velodyne)
        assert calib_times.name == number, error_msg(calib_times)
        assert label_pose.name == number, error_msg(label_pose)
        assert residual.name == number, error_msg(residual)

        new_dir = dst_dir / number
        new_dir.mkdir(parents=True, exist_ok=True)

        for dirs in [velodyne, calib_times, label_pose, residual]:
            print(f'{"=" * 100}\nCreating symbolic links for files in: {dirs}')
            for entry in dirs.iterdir():
                link_name = new_dir / entry.name
                if link_name.exists():
                    print(f"\tFile {link_name} already exists, skipping")
                    continue

                print(f"\tFile {link_name} does not exist, creating")
                link_name.symlink_to(entry, target_is_directory=entry.is_dir())

DST_DIR_MOTIONBEV = "/home/vscode/testing/methods/learning-based/MotionSeg3D/data/sequences"

dst_folder = pathlib.Path(DST_DIR_MOTIONBEV)
dst_folder.mkdir(parents=True, exist_ok=True)

create_symbolic_links(dst_dir=DST_DIR_MOTIONBEV)

In [ ]:
directory_tree.display_tree(DST_DIR_MOTIONBEV, max_depth=2)

## 4DMOS

According to the documentation, the structure should look like this:

```
./
└── sequences
  ├── 00/           
  │   ├── velodyne/	
  |   |	├── 000000.bin
  |   |	├── 000001.bin
  |   |	└── ...
  │   └── labels/ 
  |       ├── 000000.label
  |       ├── 000001.label
  |       └── ...
  ├── 01/ # 00-10 for training
  ├── 08/ # for validation
  ├── 11/ # 11-21 for testing
  └── ...
```

In [ ]:
data_path = pathlib.Path("/home/vscode/data")

SRC_DIR_VELODYNE = data_path / "kitti/SemanticKITTI/data_velodyne/sequences"
SRC_DIR_CALIB_TIMES = data_path / "kitti/SemanticKITTI/data_calib_times/sequences"
SRC_DIR_LABLE_POSE = data_path / "kitti/SemanticKITTI/data_lable_poses/sequences"
SRC_DIR_RESIDUALS = data_path / "kitti/SemanticKITTI/motionseg3d/outputs/sequences"

def create_symbolic_links(
    dst_dir: str,
    src_dir_velodyne: str = SRC_DIR_VELODYNE,
    src_dir_calib_times: str = SRC_DIR_CALIB_TIMES,
    src_dir_lable_pose: str = SRC_DIR_LABLE_POSE,
):
    sequences, dst_dir = valid_directories(
        src_dir1=src_dir_velodyne,
        src_dir2=src_dir_calib_times,
        src_dir3=src_dir_lable_pose,
        dst_dir=dst_dir,
    )

    for i, (velodyne, calib_times, label_pose) in enumerate(sequences):
        number = f"{i:02}"
        error_msg = lambda x: f"Should be: {number} is {x.name}"
        assert velodyne.name == number, error_msg(velodyne)
        assert calib_times.name == number, error_msg(calib_times)
        assert label_pose.name == number, error_msg(label_pose)

        new_dir = dst_dir / number
        new_dir.mkdir(parents=True, exist_ok=True)

        for dirs in [velodyne, calib_times, label_pose]:
            print(f'{"=" * 100}\nCreating symbolic links for files in: {dirs}')
            for entry in dirs.iterdir():

                link_name = new_dir / entry.name
                if link_name.exists():
                    print(f"\tFile {link_name} already exists, skipping")
                    continue

                print(f"\tFile {link_name} does not exist, creating")
                link_name.symlink_to(entry, target_is_directory=entry.is_dir())


DST_DIR_MOTIONBEV = data_path / "kitti/SemanticKITTI/symlinks/4dmos/sequences"

dst_folder = pathlib.Path(DST_DIR_MOTIONBEV)
dst_folder.mkdir(parents=True, exist_ok=True)

create_symbolic_links(dst_dir=DST_DIR_MOTIONBEV)

In [ ]:
directory_tree.display_tree(DST_DIR_MOTIONBEV, max_depth=2)